In [87]:
import csv 
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import numpy as np
import nltk
import itertools
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/kasem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kasem/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [88]:
def pre_process_df_keywords(df):
    for column in df.columns:
        for indx, entry in enumerate(df[column].dropna(axis=0)):
            if len(entry.split()) > 1 and entry[-1] != " ":
                df[column][indx] = entry + " "
                
def pre_process(message):
    stopWords = set(stopwords.words('english'))
    pronouns = ["we", "We","WE","I","me","Me","ME","THEY","they","They","Them","them","THEM","YOU","You","you"]
    result = []
    for word in message.split():
        if word not in stopWords and word not in pronouns:
            result.append(word)
    return (result)
    
def gen_len(iter):
    return sum([1 for _ in iter])


def isMatch_1_word(word1,word2,thresh=80): return (fuzz.ratio(word1, word2) > thresh)
def isMatch_many_words(word1,message,thresh=90): return (fuzz.partial_ratio(word1, message) >= thresh)

def column_summary(word1, df):
    summary = np.zeros(len(df.columns))
    for col_indx, worker in enumerate(df.columns):
        for word2 in df[worker].dropna(axis=0):
            if isMatch_1_word(word1,word2):
                summary[col_indx]+=1
    return summary

def column_summary_2(raw_message,df):
    summary = np.zeros(len(df.columns))
    for col_indx, worker in enumerate(df.columns):
        two_words = df[df[worker].apply(lambda x: len(str(x).split())>1)][worker]
        for word in two_words.dropna(axis=0):
            if isMatch_many_words(word, raw_message): 
                summary[col_indx]+=1
    return summary

In [89]:
xls = pd.ExcelFile('strings stems.xlsx')
first = pd.read_excel(xls, 'first').dropna(axis=1, how='all').dropna(axis=0, how='all')
parent_second = pd.read_excel(xls, 'parent second').dropna(axis=1, how='all').dropna(axis=0, how='all')
bad_keywords= pd.read_excel(xls, 'bad keywords').dropna(axis=1, how='all').dropna(axis=0, how='all')
at_least_another= pd.read_excel(xls, 'at least another').dropna(axis=1, how='all').dropna(axis=0, how='all')

pre_process_df_keywords(first)
pre_process_df_keywords(parent_second)
pre_process_df_keywords(bad_keywords)



In [57]:
def step_1(message, bad_keywords):
    one_words = bad_keywords[bad_keywords['name'].apply(lambda x: not len(x.split())>1)]["name"]
    two_words = bad_keywords[bad_keywords['name'].apply(lambda x: len(x.split())>1)]["name"]
    
    matched_words = []
    for word in pre_process(message):
        if any([isMatch_1_word(word, word2, thresh=85) for word2 in one_words]): matched_words.append(word)
    for word in two_words:
        if isMatch_many_words(word, message): matched_words.append(word)
    
    return matched_words


def step_2(message, at_least_another):
    msg = pre_process(message)
    masks = {"match": [],
             "message_indx":[],
            "df_indx": [],}

    for word_indx, word in enumerate(msg): 
        for keyword_indx, keyword in enumerate(at_least_another["name"]):
            if isMatch_1_word(word,keyword):
                masks["match"].append(word)
                masks["message_indx"].append(word_indx)
                masks["df_indx"].append(keyword_indx)



    key_word_match_rows = at_least_another.iloc[masks["df_indx"]].dropna(axis=1, how="all")
    key_word_match_rows.drop("name", axis=1)



    matching_keys = pd.DataFrame(masks)
    key_word_match_rows.drop("name", axis=1)

    words_to_drop = []
    for row_indx, row in key_word_match_rows.drop("name", axis=1).iterrows():
        if all(pd.isna(row.values)):
            words_to_drop.append(row_indx)
            continue
                
        else: words_to_drop.append(0.01)
            #for keyword in row.values:
            #    if pd.isna(keyword): continue
            #    elif any([isMatch_1_word(keyword, msg_word) for msg_word in msg]):
            #        words_to_drop.append(0.0)
            #        break
            #    else:
            #        if row_indx not in words_to_drop: words_to_drop.append(row_indx)
    words_to_drop_1 = matching_keys["match"][matching_keys["df_indx"].values == words_to_drop].values
    for i in set(words_to_drop_1):
        msg.remove(i)
    return msg

In [122]:
def summarize(message, first):
    mat = np.zeros((len(message),len(first.columns)))
    for word_indx, word in enumerate(message):    
        mat[word_indx]=(column_summary(word,first)) 
    return mat

def find_workers(message, first):
    mat = np.zeros((len(message),len(first.columns)))
    for word_indx, word in enumerate(message):    
        mat[word_indx]=(column_summary(word,first))
    
    mat = summarize(message, first)
    summary = np.array(mat).sum(axis=0)
    
    word_to_omit = []
    for word_indx, row_word in enumerate(mat[:]):
        if row_word.sum() == 0:
            word_to_omit.append(message[word_indx])
            continue
        if len(first.columns[row_word != 0]) > 1:
            word_to_omit.append(message[word_indx])

    mat = summarize(message, first)
    print(mat)
    summary = np.array(mat).sum(axis=0)
    print(summary)
    for i in word_to_omit:
        message.remove(i)

    mat = summarize(message, first)
    
    summary = np.array(mat).sum(axis=0)
    match_worker = first.columns[summary>1]
    print(f"""He is looking for >>> {match_worker}""")
    return match_worker

In [131]:
raw_message = """
I photoshop and photo shop a  ASAP. I need ASAP photoshop  vfx
""".lower()
print("Breaking") if step_1(raw_message, bad_keywords) else False
print(step_1(raw_message, bad_keywords))
message = pre_process(raw_message)

find_workers(message, first)

[]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[3. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
He is looking for >>> Index(['a graphic designer'], dtype='object')


Index(['a graphic designer'], dtype='object')

In [80]:
fuzz.ratio("photoshop","photoshoot")

84

In [86]:
[1,2,3] + [4,5,6]

[1, 2, 3, 4, 5, 6]